<a href="https://colab.research.google.com/github/taravskayavm/brain_cell_types/blob/main/TaravskayaVM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данном проекте вам необходимо создать модель для классификации типов клеток мозга по данным scRNAseq. Данные можно скачать [здесь](https://download.brainimagelibrary.org/cf/1c/cf1c1a431ef8d021/processed_data/). Нас интересует файл с экспрессиями генов (`counts.h5ad`) и с аннотацией клеток (`cell_labels.csv`).

* 3 класса
* 280186 клеток х 254 гена (данные нормализованы и немного предобработаны)
* ~ 1.0 Gb

Для работы с таким форматом данных вам необходимо будет воспользоваться библиотекой `scanpy` (Single Cell ANalysis PYthon). Ее нужно будет установить даже если вы работаете в Google Colab:

```bash
!pip install scanpy
```

Чтобы считать скачанные данные можно воспользоваться следующим кодом:

```Python
import scanpy as sc
import pandas as pd


cell_labels = pd.read_csv("PATH_TO_CELL_LABELS.csv") # Метки классов (колонка `class_label`)
adata = sc.read_h5ad("PATH_TO_COUNTS.h5ad") # Экспрессии + метаданные

adata.obs # Индексы клеток
adata.var # Гены
adata.X # Матрица экспрессий
```

Дальше вам нужно написать классификатор. Подобрать гиперпараметры, признаки и провалидировать его.

**В результате** у вас должен получиться ноутбук с описанием ваших шагов, с тренировкой и тестированием моделей, а также выводы.

In [ ]:
#!pip install scanpy

In [ ]:
# импортируем необходимые библиотеки

import scanpy as sc
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Считываем данные

cell_labels = pd.read_csv("C:/Users/med/ML/cell_labels.csv") # Метки классов (колонка `class_label`)
adata = sc.read_h5ad("C:/Users/med/ML/counts.h5ad") # Экспрессии + метаданные

In [ ]:
# Атрибуты объекта adata из библиотеки Scanpy, которая используется для анализа данных одной клетки (scRNAseq data)

adata.obs # Индексы клеток
adata.var # Гены
adata.X # Матрица экспрессий, которая содержит числовые данные об экспрессии генов в каждой клетке

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 6.1383426e-01,
        3.8716495e-02, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 6.1811280e-01,
        5.1406868e-02, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 4.4882911e-01,
        4.1903242e-02, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 5.6548023e+00,
        4.2430919e-02, 4.5176870e-03],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.1598111e+01,
        2.7926207e-01, 7.2065614e-02],
       [0.0000000e+00, 4.7502649e-01, 0.0000000e+00, ..., 1.4223028e+01,
        3.1781629e-01, 1.0914283e+00]], dtype=float32)

**Анализируем наши данные**

In [ ]:
cell_labels.head()

,Unnamed: 0,sample_id,slice_id,class_label,subclass,label
0,10000143038275111136124942858811168393,mouse2_sample4,mouse2_slice31,Other,Astro,Astro_1
1,100001798412490480358118871918100400402,mouse2_sample5,mouse2_slice160,Other,Endo,Endo
2,100006878605830627922364612565348097824,mouse2_sample6,mouse2_slice109,Other,SMC,SMC
3,100007228202835962319771548915451072492,mouse1_sample2,mouse1_slice71,Other,Endo,Endo
4,100009332472089331948140672873134747603,mouse2_sample5,mouse2_slice219,Glutamatergic,L2/3 IT,L23_IT_3


In [ ]:
cell_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280186 entries, 0 to 280185
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Unnamed: 0   280186 non-null  object
 1   sample_id    280186 non-null  object
 2   slice_id     280186 non-null  object
 3   class_label  280186 non-null  object
 4   subclass     280186 non-null  object
 5   label        280186 non-null  object
dtypes: object(6)
memory usage: 12.8+ MB


In [ ]:
cell_labels["class_label"].unique()

array(['Other', 'Glutamatergic', 'GABAergic'], dtype=object)

In [ ]:
cell_labels["class_label"].value_counts(normalize=True)

class_label
Glutamatergic    0.556170
Other            0.378516
GABAergic        0.065314
Name: proportion, dtype: float64

Пропорции классов немного отличаются друг от друга. Класс Glutamatergic преобладает с заметным перевесом (55,6%), в то время как класс GABAergic является наименее представленным (6,5%). Однако, учитываея ограниченность ресурсов для выборки данных модель будем обучать **без балансировки классов**.

**Подготовка данных**

Предоставленные данные уже нормализованы и предобработаны, поэтому просто **разделяем данные на обучающий и тестовый наборы:**

test_size=0.2: Этот параметр указывает на то, какую долю данных мы отводим для тестирования (в данном случае 20%). Это стандартное соотношение часто используется для разделения данных на обучающий и тестовый наборы. 80% данных используются для обучения модели, а 20% для тестирования, чтобы оценить ее производительность.

random_state=42: Этот параметр используется для установления начального состояния генератора случайных чисел. При таком фиксированном значении random_state, разделение данных всегда будет одинаковым, что позволяет повторяемо проверять модель.

In [ ]:
X = adata.X                       # признаки
y = cell_labels['class_label']    # целевая переменная
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Данные готовы.")


Данные готовы.


**Обучение модели Random Forest**

При обучении Random Forest каждое дерево строится на случайной подвыборке образцов данных и случайном подмножестве признаков.
Каждое дерево учится на своем поднаборе данных и потом применяется для предсказания классов.
В итоге, предсказания каждого дерева объединяются (через голосование в случае классификации) для получения окончательного прогноза модели.

Создаем объект класса **RandomForestClassifier** с двумя гиперпараметрами:

**n_estimators=100**: этот гиперпараметр указывает на количество деревьев в лесу случайных деревьев (Random Forest). Обычно чем больше деревьев, тем лучше качество предсказаний модели. Значение 100 является распространенным выбором. Это дает хороший баланс между качеством модели и вычислительной эффективностью;

**random_state=42**: гиперпараметр используется для установления начального состояния генератора случайных чисел в модели. Это необходимо для обеспечения воспроизводимости результатов обучения модели. При установке random_state на фиксированное значение (например, 42), предсказания модели будут одинаковыми при каждом запуске обучения на одних и тех же данных.


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print("Модель обучена.")

Модель обучена.


**Оценка модели**

In [ ]:
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

               precision    recall  f1-score   support

    GABAergic       1.00      0.98      0.99      3636
Glutamatergic       1.00      0.99      1.00     31266
        Other       0.99      1.00      0.99     21136

     accuracy                           0.99     56038
    macro avg       1.00      0.99      0.99     56038
 weighted avg       0.99      0.99      0.99     56038

[[ 3570    15    51]
 [    1 31102   163]
 [    4    56 21076]]


**Анализ результатов**
- **precision (точность):** высокие значения близкие к 1 для всех классов указывают на то, что модель редко дает ложноположительные результаты;
- **recall (полнота):**  высокие значения близкие к 1 означают, что модель хорошо находит истинно положительные результаты;
- **f1-score (среднее гармоническое между точностью и полнотой):** показывает высокую сбалансированность модели по всем классам;
- **матрица ошибок (confusion matrix)**: показывает количество верно и неверно классифицированных образцов для каждого класса (строки - фактические классы, столбцы - предсказанные классы). В нашем случае матрица ошибок показывает, что основной диагонали соответствуют правильные классификации, а внедиагональные элементы представляют ошибки классификации, которые в целом незначительны.

**Выводы**

- модель Random Forest хорошо справилась с задачей классификации типов клеток мозга по данным scRNAseq. Ее производительность характеризуется высокими значениями точности (precision), полноты и F1-меры для всех классов;
- модель обладает высокой обобщающей способностью и хорошо интерпретирует различия между типами клеток на основе экспресси генов.

**Сохраняем модель в текущем рабочем каталоге**

In [ ]:
joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']